In [2]:
import torch
from ogb.utils import smiles2graph
from torch_geometric.data import Data
from model_nn import MPNN, MCULE_DATA
import pytorch_lightning as pl
import numpy as np
import random
from deepchem.splits import RandomSplitter


torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)
random.seed(0)


dataset = MCULE_DATA('./datasets/').shuffle()

splitter = RandomSplitter()
train_idx, valid_idx, test_idx = splitter.split(dataset,frac_train=0.7, frac_valid=0.1, frac_test=0.2)
train_dataset = dataset[list(train_idx)]
valid_dataset = dataset[list(valid_idx)]
test_dataset = dataset[list(test_idx)]

mean = dataset.data.y.mean()
std = dataset.data.y.std()


c:\Users\loicb\miniconda3\lib\site-packages\torch_geometric\data\in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The given 'InMemoryDataset' only references a subset of examples of the full dataset, but 'data' will contain information of the full dataset. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [4]:
model = MPNN(
    hidden_dim=80,
    out_dim=1,
    std=std,
    train_data=train_dataset,
    valid_data=valid_dataset,
    test_data=test_dataset,
    lr=0.001,
    batch_size=64
)

model.load_state_dict(torch.load('gnn_model.pth'))
model.eval()


smiles='c1ccccc1'
graph = smiles2graph(smiles)

In [5]:
x = torch.tensor(graph['node_feat'], dtype=torch.long)
edge_index = torch.tensor(graph['edge_index'], dtype=torch.long)
edge_attr = torch.tensor(graph['edge_feat'], dtype=torch.long)
data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

with torch.no_grad():
    output = model(data)

print(output)

tensor([53.5123])
